In [1]:
import Ipynb_importer
import dzdp_common
from dzdp_common import *
from multiprocessing.dummy import Pool as ThreadPool

importing Jupyter notebook from dzdp_common.ipynb
importing Jupyter notebook from IPpool.ipynb


In [2]:
crawl = Crawl_common(more=True)

获取ip池----------------------
获取4个网站的ip
一共获取了3个ip


## 1.分类层

In [5]:
def classify(city_url,city,big_class):
    soup = crawl.requests_visit_url(city_url,is_use_ip=True)
    clf_ls = []
    for i in soup.select("#classfy a"):
        clf_url = i["href"]
        clf_tl = i.text
        clf_ls.append([big_class,city,clf_tl,clf_url])
    return clf_ls

In [8]:
clf_ls = classify("http://www.dianping.com/search/category/4/80","广州","生活服务")

0
{'https': 'https://123.163.166.104:23817'}


In [9]:
clf_ls

[['生活服务', '广州', '家政', 'http://www.dianping.com/search/category/4/80/g195'],
 ['生活服务', '广州', '生活配送', 'http://www.dianping.com/search/category/4/80/g2928'],
 ['生活服务', '广州', '房屋地产', 'http://www.dianping.com/search/category/4/80/g836'],
 ['生活服务', '广州', '便民服务', 'http://www.dianping.com/search/category/4/80/g33971'],
 ['生活服务', '广州', '金融', 'http://www.dianping.com/search/category/4/80/g34003'],
 ['生活服务', '广州', '搬家运输', 'http://www.dianping.com/search/category/4/80/g33986'],
 ['生活服务', '广州', '文印图文', 'http://www.dianping.com/search/category/4/80/g3066'],
 ['生活服务', '广州', '快照摄影', 'http://www.dianping.com/search/category/4/80/g3064'],
 ['生活服务',
  '广州',
  '家电数码维修',
  'http://www.dianping.com/search/category/4/80/g33976'],
 ['生活服务', '广州', '洗涤护理', 'http://www.dianping.com/search/category/4/80/g33762'],
 ['生活服务', '广州', '商务服务', 'http://www.dianping.com/search/category/4/80/g33958'],
 ['生活服务', '广州', '居家维修', 'http://www.dianping.com/search/category/4/80/g26117'],
 ['生活服务', '广州', '旅行社', 'http://www.dianping

## 2.三级分类层

In [4]:
def sub_classify(clf_ls):
    url = clf_ls[-1]  #获取url
    sub_clf_ls = []  #获取子分类的url
    soup = crawl.requests_visit_url(url,is_use_ip=True) # 获取soup
    
    #*******如果存在子分类*********
    if soup.select('#classfy-sub'):
        for i in soup.select('#classfy-sub a'):
            if i.text != '不限': #过滤掉字符属性为不限的url
                sub_clf_url = i["href"]  #子分类的url
                sub_clf_tl = i.text      #子分类的title
                temp_ls = clf_ls[:-1]
                temp_ls.extend([sub_clf_tl,sub_clf_url]) #添加子分类
                sub_clf_ls.append(temp_ls)      #将结果装起来
        return sub_clf_ls
    
    #*****如果不存在*********
    else:
        print("{0}没有子分类".format(clf_ls[-2]))
        clf_ls.insert(3,"没有三级分类")
        sub_clf_ls.append(clf_ls)
        return sub_clf_ls
     

In [79]:
sub_classify(['生活服务', '广州', '生活配送', 'http://www.dianping.com/search/category/4/80/g2928'])

0
{'https': 'https://49.87.75.31:30827'}


[['生活服务',
  '广州',
  '生活配送',
  '送水站',
  'http://www.dianping.com/search/category/4/80/g34011'],
 ['生活服务',
  '广州',
  '生活配送',
  '送奶站',
  'http://www.dianping.com/search/category/4/80/g34012']]

### 3、区县层

In [21]:
soup = crawl.requests_visit_url("http://www.dianping.com/search/category/4/80/g2980",is_use_ip=True)

0
{'https': 'https://115.210.72.148:29356'}


In [23]:
soup.select("#region-nav a")

[<a href="http://www.dianping.com/search/category/4/80/g2980r22"><span>天河区</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g2980r24"><span>越秀区</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g2980r25"><span>海珠区</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g2980r27"><span>白云区</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g2980r621"><span>番禺区</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g2980r26"><span>荔湾区</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g2980r1552"><span>花都区</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g2980r1554"><span>增城区</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g2980r1549"><span>黄埔区</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g2980r1964"><span>萝岗区</span></a>]

In [5]:
def get_region(sub_clf_ls=None):
    if sub_clf_ls[-2]=="没有三级分类":
        print('正在爬取{0}'.format(sub_clf_ls[-3]))
    else:
        print('正在爬取{0}'.format(sub_clf_ls[-2]))
    url = sub_clf_ls[-1]  #url
    soup = crawl.requests_visit_url(url,is_use_ip=True) #访问
    region_ls = [] #地区的列表
    
    #**********如果存在区县
    if soup.select("#region-nav a"):
        for i in soup.select("#region-nav a"):
            if "不限" not in i.text.strip():
                region_url =i["href"] 
                region_tl = i.text.strip()
                region = sub_clf_ls[:-1]
                region.extend([region_tl,region_url])
                region_ls.append(region)
                
        return region_ls
    
    #*******如果不存在*********
    else:
        print("没有行政区")
        sub_clf_ls.insert(4,"没有行政区")
        region_ls.append(sub_clf_ls)
        return region_ls
 

In [76]:
get_region(['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  'http://www.dianping.com/search/category/4/80/g33959'])

正在爬取律师服务
0
{'https': 'https://59.40.51.183:8010'}


[['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  'http://www.dianping.com/search/category/4/80/g33959r22'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '越秀区',
  'http://www.dianping.com/search/category/4/80/g33959r24'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '海珠区',
  'http://www.dianping.com/search/category/4/80/g33959r25'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '白云区',
  'http://www.dianping.com/search/category/4/80/g33959r27'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '番禺区',
  'http://www.dianping.com/search/category/4/80/g33959r621'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '荔湾区',
  'http://www.dianping.com/search/category/4/80/g33959r26'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '花都区',
  'http://www.dianping.com/search/category/4/80/g33959r1552'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '增城区',
  'http://www.dianping.com/search/category/4/80/g33959r1554'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '从化区',
  'http://www.dianping.com/search/category/4/80/g33959r1555'],
 ['生活服务',
  '广州',
  '

### 4.商圈层

In [32]:
soup = crawl.requests_visit_url("http://www.dianping.com/search/category/4/80/g3064r1963",is_use_ip=True)

0
{'https': 'https://180.173.210.189:53281'}


In [34]:
soup.select("#region-nav-sub a")

[<a class="cur" href="http://www.dianping.com/search/category/4/80/g3064r1963"><span>不限</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g3064r27441"><span>金州商业区</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g3064r27446"><span>珠江旧镇</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g3064r27445"><span>大岗</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g3064r30188"><span>东涌</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g3064r27443"><span>华汇广场</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g3064r64981"><span>万顷沙</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g3064r27442"><span>黄阁</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g3064r64979"><span>榄核镇</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g3064r64982"><span>鱼窝头</span></a>,
 <a href="http://www.dianping.com/search/category/4/80/g3064r64980"><span>大涌</span></a>]

In [6]:
def get_circle(region_ls=None):
    url = region_ls[-1] #url
    circle_ls =[]
    
    #******爬取网页***********
    soup = crawl.requests_visit_url(url,is_use_ip=True)
    
    #*****如果存在***********
    if soup.select("#region-nav-sub a"):
        for i in soup.select("#region-nav-sub a"):
            if i.text.strip()!="不限":
                circle_url = i["href"]  #url
                circle_tit = i.text     #商圈名
                circle = region_ls[:-1]  
                circle.extend([circle_tit,circle_url]) 
                circle_ls.append(circle)
        return circle_ls      
    
    #*****如果不存在***********
    else:
        print("商圈不存在！！")
        region_ls.insert("没有商圈")
        circle_ls.append(region_ls)
        return circle_ls

In [77]:
get_circle(['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  'http://www.dianping.com/search/category/4/80/g33959r22'])

0
{'https': 'https://60.255.186.169:8888'}


[['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  '天河城/体育中心',
  'http://www.dianping.com/search/category/4/80/g33959r1519'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  '珠江新城',
  'http://www.dianping.com/search/category/4/80/g33959r1522'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  '石牌/龙口',
  'http://www.dianping.com/search/category/4/80/g33959r1521'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  '天河北',
  'http://www.dianping.com/search/category/4/80/g33959r1520'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  '兴盛路/跑马场',
  'http://www.dianping.com/search/category/4/80/g33959r30510'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  '天河公园',
  'http://www.dianping.com/search/category/4/80/g33959r1523'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  '车陂/东圃',
  'http://www.dianping.com/search/category/4/80/g33959r2080'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  '天河其他',
  'http://www.dianping.com/search/category/4/80/g33959r1525']]

### 5.商家列表层

In [3]:
#获取每个商家的链接、店名
def get_stores(circle_ls = None):
    store_ls=[] #存储的列表
    url = circle_ls[-1]  #url
    soup = crawl.requests_visit_url(url,is_use_ip=True,timeout=5)  #访问当前主页
    is_continue = True
    page = 0
    while is_continue:
        page += 1
        print("正在爬取第{0}页".format(page))
        
        #解析数据
        if soup.select(".pic a")==[]:
            print("此页没有任何信息！！")
            store_ls = []
            is_continue = False 
            
        else:
            for i in soup.select(".pic a"):  
                try :
                    store_name = i.img["alt"]  #酒店名
                    link = i["href"]#详情层的url
                except Exception as e:
                    print("get_stores函数报错了！！！")
                    print(e)
                    print("------------------------")
                    print("")
                    store_name ="出错了"
                    link = "出错了"
                store = circle_ls[:5]  #只取前5位
                store.extend([url,store_name,link]) 
                store_ls.append(store)

            #获取下一页的数据
            next_tag = soup.find_all("a",text = re.compile("下一页"))  #获取字符属性为下一页的url
            if next_tag != []:                                         #保证含有字符属性
                url = next_tag[0]["href"]     #获取下一页的url
                print(url)
                soup = crawl.requests_visit_url(url,is_use_ip=True,timeout=5)   #更新BeautifulSoup对象
            else:
                url = ''
                is_continue  = False                                  #如果没有下一页，就停止更新
    
    return store_ls

In [78]:
get_stores(['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  '天河城/体育中心',
  'http://www.dianping.com/search/category/4/80/g33959r1519'])

0
{'https': 'https://59.40.50.190:8010'}
正在爬取第1页


[['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  'http://www.dianping.com/search/category/4/80/g33959r1519',
  '君厚律师事务所',
  'http://www.dianping.com/shop/83426191'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  'http://www.dianping.com/search/category/4/80/g33959r1519',
  '天胜律师事务所',
  'http://www.dianping.com/shop/45175702'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  'http://www.dianping.com/search/category/4/80/g33959r1519',
  '任高扬律师事务所',
  'http://www.dianping.com/shop/44488693'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  'http://www.dianping.com/search/category/4/80/g33959r1519',
  '启源律师事务所',
  'http://www.dianping.com/shop/44978341'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  'http://www.dianping.com/search/category/4/80/g33959r1519',
  '仁言律师事务所',
  'http://www.dianping.com/shop/80198805'],
 ['生活服务',
  '广州',
  '商务服务',
  '律师服务',
  '天河区',
  'http://www.dianping.com/search/category/4/80/g33959r1519',
  '天池宏范律师事务所',
  'http://www.dianping.com/shop/77784871'],
 

## 6. 主函数流程

### 6.1 获取全部的分类

In [8]:
clf_ls = classify("http://www.dianping.com/search/category/4/80","广州","生活服务")

0
{'https': 'https://180.118.241.241:61234'}


In [9]:
clf_ls

[['生活服务', '广州', '家政', 'http://www.dianping.com/search/category/4/80/g195'],
 ['生活服务', '广州', '生活配送', 'http://www.dianping.com/search/category/4/80/g2928'],
 ['生活服务', '广州', '房屋地产', 'http://www.dianping.com/search/category/4/80/g836'],
 ['生活服务', '广州', '便民服务', 'http://www.dianping.com/search/category/4/80/g33971'],
 ['生活服务', '广州', '金融', 'http://www.dianping.com/search/category/4/80/g34003'],
 ['生活服务', '广州', '搬家运输', 'http://www.dianping.com/search/category/4/80/g33986'],
 ['生活服务', '广州', '文印图文', 'http://www.dianping.com/search/category/4/80/g3066'],
 ['生活服务', '广州', '快照摄影', 'http://www.dianping.com/search/category/4/80/g3064'],
 ['生活服务',
  '广州',
  '家电数码维修',
  'http://www.dianping.com/search/category/4/80/g33976'],
 ['生活服务', '广州', '洗涤护理', 'http://www.dianping.com/search/category/4/80/g33762'],
 ['生活服务', '广州', '商务服务', 'http://www.dianping.com/search/category/4/80/g33958'],
 ['生活服务', '广州', '居家维修', 'http://www.dianping.com/search/category/4/80/g26117'],
 ['生活服务', '广州', '旅行社', 'http://www.dianping

### 6.2 获取三级分类

In [10]:
%%time
sub_clf_ls = list(map(sub_classify,clf_ls))

0
{'https': 'https://115.202.254.19:42229'}
0
{'https': 'https://59.49.76.53:808'}
0
{'https': 'https://101.71.194.178:53281'}
0
{'https': 'https://171.212.141.50:8118'}
0
{'https': 'https://180.118.241.241:61234'}
0
{'https': 'https://121.226.162.172:47239'}
0
{'https': 'https://27.40.132.14:61234'}
0
{'https': 'https://121.226.162.172:47239'}
快照摄影没有子分类
0
{'https': 'https://180.173.210.189:53281'}
0
{'https': 'https://60.255.186.169:8888'}
0
{'https': 'https://175.17.14.241:8080'}
0
{'https': 'https://59.49.76.53:808'}
0
{'https': 'https://124.234.240.207:8118'}
旅行社没有子分类
0
{'https': 'https://59.40.51.93:8010'}
0
{'https': 'https://121.31.196.43:8123'}
0
{'https': 'https://36.26.222.126:808'}
公司企业没有子分类
0
{'http': 'http://125.72.75.94:8110'}
0
{'https': 'https://36.26.222.126:808'}
演出票务没有子分类
0
{'https': 'https://59.40.51.93:8010'}
0
{'https': 'https://221.205.182.97:80'}
商圈没有子分类
0
{'http': 'http://59.111.97.62:80'}
访问被拒
1
{'https': 'https://36.26.222.126:808'}
政府机构没有子分类
0
{'https': 'htt

In [11]:
sub_clf_ls = list(itertools.chain.from_iterable(sub_clf_ls))

### 6.3 获取区县

In [32]:
%%time
region_ls = []
async_pool = ThreadPool(processes=100)
result = async_pool.map_async(get_region,sub_clf_ls)
async_pool.close()
async_pool.join()
region_ls.append(result.get())
print("***********")
region_ls = list(itertools.chain.from_iterable(region_ls))
region_ls = list(itertools.chain.from_iterable(region_ls))
print("------------")

正在爬取保姆陪护正在爬取其他家政正在爬取送水站正在爬取送奶站正在爬取小区正在爬取商务楼正在爬取房产中介正在爬取保洁清洗正在爬取公寓正在爬取花店正在爬取通信营业厅正在爬取医院正在爬取厕所正在爬取齿科正在爬取社区服务正在爬取生活缴费正在爬取体检中心正在爬取邮局正在爬取书刊订阅正在爬取银行正在爬取保险公司正在爬取证券投资正在爬取典当行正在爬取拍卖行正在爬取其他金融正在爬取物流快递正在爬取搬家正在爬取广告印刷正在爬取复印刻章正在爬取其他文印图文正在爬取快照摄影正在爬取家电维修正在爬取电脑维修正在爬取手机维修正在爬取数码维修正在爬取彩票正在爬取裁缝改衣正在爬取修鞋擦鞋正在爬取其他洗涤护理正在爬取律师服务正在爬取咨询服务正在爬取财务服务正在爬取人才服务正在爬取设备维修正在爬取摩托车维修正在爬取电动/自行车维修正在爬取开锁换锁正在爬取家装维修正在爬取钟表维修正在爬取房屋防水正在爬取管道疏通正在爬取家具维修正在爬取其他居家维修正在爬取旅行社正在爬取传媒服务正在爬取电视广播正在爬取出版社正在爬取报刊杂志社正在爬取废品回收正在爬取金属回收正在爬取金银首饰回收正在爬取家具家电回收正在爬取奢侈品回收正在爬取其他回收正在爬取信用社正在爬取公司企业正在爬取飞机票代售点正在爬取火车票代售点正在爬取长途汽车票代售点正在爬取景点售票处正在爬取其他家电数码维修正在爬取船票代售点正在爬取翻译服务正在爬取其他售票点正在爬取演出票务正在爬取地铁/轻轨正在爬取公交车正在爬取火车站正在爬取长途汽车站正在爬取洗衣店正在爬取港口/码头正在爬取飞机场正在爬取其他公共交通正在爬取商圈正在爬取政府机构正在爬取网站正在爬取机场休息室正在爬取丧葬设施正在爬取成人用品有人店正在爬取白事服务正在爬取成人用品无人店正在爬取数码回收正在爬取更多生活服务




























































































000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000




























































### 6.4 获取商圈

In [35]:
%%time
circle_ls= []
async_pool = ThreadPool(processes=100)
result = async_pool.map_async(get_region,region_ls)
async_pool.close()
async_pool.join()
circle_ls.append(result.get())

正在爬取天河区正在爬取白云区正在爬取花都区正在爬取黄埔区正在爬取白云区正在爬取花都区正在爬取越秀区正在爬取番禺区正在爬取黄埔区正在爬取从化区正在爬取越秀区正在爬取番禺区正在爬取增城区正在爬取萝岗区正在爬取越秀区正在爬取番禺区正在爬取荔湾区正在爬取天河区正在爬取白云区正在爬取黄埔区正在爬取天河区正在爬取白云区正在爬取黄埔区正在爬取白云区正在爬取花都区正在爬取黄埔区正在爬取花都区正在爬取天河区正在爬取黄埔区正在爬取天河区正在爬取白云区正在爬取花都区正在爬取黄埔区正在爬取花都区正在爬取黄埔区正在爬取白云区正在爬取白云区正在爬取花都区正在爬取黄埔区正在爬取天河区正在爬取黄埔区正在爬取天河区正在爬取白云区正在爬取花都区正在爬取黄埔区正在爬取天河区正在爬取白云区正在爬取花都区正在爬取黄埔区正在爬取天河区正在爬取天河区正在爬取白云区正在爬取花都区正在爬取花都区正在爬取花都区正在爬取黄埔区正在爬取天河区正在爬取天河区正在爬取白云区正在爬取花都区正在爬取黄埔区正在爬取天河区正在爬取白云区正在爬取越秀区正在爬取黄埔区正在爬取番禺区正在爬取增城区正在爬取白云区正在爬取南沙区正在爬取花都区正在爬取海珠区正在爬取荔湾区正在爬取白云区正在爬取从化区正在爬取白云区正在爬取天河区正在爬取南沙区正在爬取海珠区正在爬取荔湾区正在爬取从化区正在爬取南沙区正在爬取海珠区正在爬取天河区正在爬取荔湾区正在爬取花都区正在爬取从化区正在爬取南沙区正在爬取海珠区正在爬取荔湾区正在爬取黄埔区正在爬取黄埔区正在爬取花都区正在爬取从化区正在爬取番禺区正在爬取天河区正在爬取海珠区正在爬取天河区正在爬取白云区正在爬取花都区正在爬取南沙区



































































































0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


































































































{

In [37]:
circle_ls = list(itertools.chain.from_iterable(circle_ls))

In [40]:
circle_ls = list(itertools.chain.from_iterable(circle_ls))

In [5]:
crawl.get_spc_path("/home/zhh/dzdp_data/","生活服务")

done!!


In [5]:
crawl.save_cache(circle_ls,"生活服务_circle.pickle")

NameError: name 'circle_ls' is not defined

In [6]:
circle_ls = crawl.read_cache("生活服务_circle.pickle")

In [7]:
chunkes = crawl.get_chunk(100,circle_ls)

In [8]:
len(chunkes)

97

### 6.5 获取商家列表

In [ ]:
%%time
if __name__=="__main__":
    store_ls=[]
    offset = 13
    #********每个chunk去爬取*************
    for index,chunk in enumerate(chunkes[offset:]):
        #***********一千个进程***************
        async_pool = ThreadPool(processes=150)
        result = async_pool.map_async(get_stores,chunk)
        async_pool.close()
        async_pool.join()
        print("第{0}个chunk已经爬取完毕！\n".format(index+offset))
        result1 = result.get()
        result2 = list(itertools.chain.from_iterable(result1))
        store_ls.append(result1)
        time.sleep(10)
        #新建立一个子文件
        crawl.get_spc_path("/home/zhh/dzdp_data/生活服务/","广州")
        crawl.save_cache(result2,"生活服务_广州_store_ls_{0}".format(index+offset))

----------
----------
----------
----------
----------
----------
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r1963p2
----------
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r1964p2
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r1554p2
正在爬取第2页
此页没有任何信息！！
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r1555p2
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r24p2
----------
正在爬取第1页
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r1964p2
http://www.dianping.com/search/category/4/80/g835r26p2
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r24p2
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r26p2
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r1552p2
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r1554p2
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r26p2
正在爬取第1页
正在爬取第1页
http://www.dianping.com/search/category/4/80/g835r1552p2
http://www.dianping.com/search/category/4/80/g835r